In [1]:
import pymysql
import pandas as pd
import numpy as np
from pydataset import data
from get_db_url import make_db



In [10]:
make_db('employees', 'SELECT * FROM departments LIMIT 10')

,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance
3,d003,Human Resources
4,d001,Marketing
5,d004,Production
6,d006,Quality Management
7,d008,Research
8,d007,Sales


In [11]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [12]:
#1. Copy the users and roles DataFrames from the examples above.

# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [13]:
#2. What is the result of using a right join on the DataFrames?


(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='right')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)
#output is 5x4 grid. shows all roles whether filled or not 

,id,employee,role_id,role
0,1.0,bob,1,admin
1,2.0,joe,2,author
2,3.0,sally,3,reviewer
3,4.0,adam,3,reviewer
4,NaN,NaN,4,commenter


In [15]:
#3.What is the result of using an outer join on the DataFrames?

(users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)
#result is 7x4 grid. shows all Ids and all roles 

,id,employee,role_id,role
0,1.0,bob,1.0,admin
1,2.0,joe,2.0,author
2,3.0,sally,3.0,reviewer
3,4.0,adam,3.0,reviewer
4,5.0,jane,NaN,NaN
5,6.0,mike,NaN,NaN
6,NaN,NaN,4.0,commenter


In [110]:
#4. What happens if you drop the foreign keys from the 
#DataFrames and try to merge them?

# Perform an outer join specifying the left and right DataFrame keys.

# Perform an outer join specifying the left and right DataFrame keys.

pd.merge(users, roles, 
        how='outer',
        indicator = True)


#answer: Not a good merge because of output 

,id,name,role_id,_merge
0,1,bob,1.0,left_only
1,2,joe,2.0,left_only
2,3,sally,3.0,left_only
3,4,adam,3.0,left_only
4,5,jane,NaN,left_only
5,6,mike,NaN,left_only
6,1,admin,NaN,right_only
7,2,author,NaN,right_only
8,3,reviewer,NaN,right_only
9,4,commenter,NaN,right_only


In [112]:
#5. Load the mpg dataset from PyDataset.
from pydataset import data 
mpg = data('mpg')


In [29]:
#6. Output and read the documentation for the mpg dataset.
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [30]:
#7. How many rows and columns are in the dataset?
mpg.shape

#234 rows/ 11 columns 

(234, 11)

In [114]:
#8. Check out your column names and perform any 
#cleanup you may want on them.
mpg.columns



Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [85]:
#9 Display the summary statistics for the dataset.
mpg.describe().round(2)

,displ,year,cyl,cty,hwy
count,234.00,234.00,234.00,234.00,234.00
mean,3.47,2003.50,5.89,16.86,23.44
std,1.29,4.51,1.61,4.26,5.95
min,1.60,1999.00,4.00,9.00,12.00
25%,2.40,1999.00,4.00,14.00,18.00
50%,3.30,2003.50,6.00,17.00,24.00
75%,4.60,2008.00,8.00,19.00,27.00
max,7.00,2008.00,8.00,35.00,44.00


In [43]:
#10 How many different manufacturers are there?
mpg.manufacturer.nunique()
#mpg.manufacturer.unique() - lisit of names 

15

In [44]:
#11 How many different models are there?
mpg.model.nunique()

38

In [116]:
#12 Create a column named mileage_difference 
#like you did in the DataFrames exercises; this column 
#should contain the difference between 
#highway and city mileage for each car.

mpg['mileage_difference'] = mpg.hwy - mpg.cty




In [119]:
#13 Create a column named average_mileage like you did 
#in the DataFrames exercises; this is the mean of the 
#city and highway mileage.

mpg['average_mileage'] = round(2/((1/mpg.hwy) + (1/mpg.cty)),2)

In [125]:
#14 Create a new column on the mpg dataset named is_automatic 
#that holds boolean values denoting whether 
#the car has an automatic transmission.

#mpg.trans.unique()- lists different category types 

mpg['is_automatic']= mpg.trans.str.contains('auto')

mpg



,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,22.21,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,24.36,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,24.31,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,24.71,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,19.81,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,22.64,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,24.36,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,19.81,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,21.27,False


In [130]:
#15 Using the mpg dataset, find out which which manufacturer 
#has the best miles per gallon on average?

mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending = False).head()

manufacturer
honda         27.891111
volkswagen    24.370000
hyundai       22.002143
subaru        21.982857
toyota        21.224706
Name: average_mileage, dtype: float64

In [136]:
#16 Do automatic or manual cars have better miles per gallon?

mpg['transmission_category'] = np.where(mpg.trans.str.contains('auto'),'automatic', 'manual')

In [137]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage,is_automatic,transmission_category
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,22.21,True,automatic
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,24.36,False,manual
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,24.31,False,manual
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,24.71,True,automatic
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,19.81,True,automatic


In [138]:
mpg.groupby('transmission_category')[['average_mileage']].mean().round()

,average_mileage
transmission_category,
automatic,19.0
manual,22.0


# Exercise pt 3 

In [ ]:
# 1. Use your get_db_url function to help you explore the data from the chipotle database.



In [25]:
orders = make_db('chipotle', 'SELECT * FROM orders')

orders

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [20]:
#2. What is the total price for each order?
make_db('chipotle','SELECT order_id, SUM(TRIM(LEADING "$" FROM item_price))as new_price FROM orders GROUP BY order_id')

#orders['item_price'] = orders.item_price.str.replace('s', '')
#order_totals = orders.groupby('order_id').item_price.sum()

,order_id,new_price
0,1,11.56
1,2,16.98
2,3,12.67
3,4,21.00
4,5,13.70
...,...,...
1829,1830,23.00
1830,1831,12.90
1831,1832,13.20
1832,1833,23.50


In [23]:
#3. What are the most popular 3 items?

make_db('chipotle','SELECT item_name,SUM(quantity) as quantity_purchased from orders GROUP BY item_name ORDER BY quantity_purchased DESC')

#orders.groupby('item').quantity.sum().sort_values(ascending = False)

,item_name,quantity_purchased
0,Chicken Bowl,761.0
1,Chicken Burrito,591.0
2,Chips and Guacamole,506.0
3,Steak Burrito,386.0
4,Canned Soft Drink,351.0
5,Chips,230.0
6,Steak Bowl,221.0
7,Bottled Water,211.0
8,Chips and Fresh Tomato Salsa,130.0
9,Canned Soda,126.0


In [ ]:
#4. Which item has produced the most revenue?
orders.groupby('item_name').item_price.sum().sort_values(ascending=False).head(1)



In [24]:
#5. Join the employees and titles DataFrames together.
make_db('employees','SELECT * FROM employees JOIN titles on employees.emp_no = titles.emp_no')

,emp_no,birth_date,first_name,last_name,gender,hire_date,emp_no,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,10002,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,10004,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,499997,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,499998,Staff,1993-12-27,1998-12-27


In [ ]:
#6. For each title, find the hire date of the 
#employee that was hired most recently with that title.
all_emp_titles.groupby('title').hire_date.max()

In [ ]:
dept_title_querry = '''
                    SELECT t.emp_no,
                    t.title,
                    t.to_date,
                    d.dept_name,
                    FROM departments AS d
                    JOIN dept_emp AS de USING(dept_no)
                    JOIN titles AS t USING(emp_no)
'''
dept_titles = pd.read_sql(dept_title_query, get_db_url('employees'))
dept_titles.head()

In [ ]:
#7. Write the code necessary to create a cross tabulation of the number 
#of titles by department. 
#(Hint: this will involve a combination of SQL code 
#to pull the necessary data 
#and python/pandas code to perform the manipulations.)    

all_titles_crosstab = pd.crosstab(dept_titles.dept_name, dept_titles.title)
